In [18]:
'''try to get all realized structures from eqg table on the field of eq. groups
try to check ready trees if they are correct'''
# input - col strings like: '1 2 3 4 5' + col eqg value

#можно еще ускорить если производить запись названий групп в буквенном виде без пробелов

import numpy as np
import pandas as pd
from itertools import product
from itertools import combinations
from itertools import chain
import string
import time
import math
import re

def listToString_s(list1):
    return (' '.join(map(str, list1)))
def str2hexstr(txt):
    t = [np.base_repr(int(i), base=16) for i in txt.split()]
    st = ''
    flag_once = 0
    for i in t:
        if flag_once:
            st = st + ' ' + i
        else:
            st += i
            flag_once = 1
    return st

num2alpha = dict(zip(range(1, 27), string.ascii_lowercase))

def digit2alpha(txt):
    '''converts string of digits into letters'''
    # add defence from input 0 and bigger than 26
    my_list = txt.split()
    txt_res = ''
    for i in range(len(my_list)):
        txt_res += num2alpha[int(my_list[i])] + ' '
    return txt_res.rstrip()

def alpha2digit(txt):
    '''converts string of letters into digits'''
    mapping = {"a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6, "g": 7, "h": 8,
           "i": 9, "j": 10, "k": 11, "l": 12, "m": 13, "n": 14, "o": 15,
           "p": 16, "q":17,"r":18, "s": 19, "t": 20,"u": 21, "v": 22, "w": 23, "x": 24, "y": 25, "z":26}
    for k,v in mapping.items():
        v = str(v)
        txt = txt.replace(k,v)
    return txt

def is_uppcase_in_shell(text):
    #flag_up = 0
    flag_low = 0
    for i in range(len(text)):
        if text[i] == '(':
            flag_low = 0
            continue
        if text[i].islower():
            flag_low = 1
            continue
        if text[i] == ')' and flag_low == 0:
            #flag_up = 1
            #break
            return 1
    #if flag_up: 
    return 0    

def braket2ml(text): 
# не работает для двузначных
    txt = ''
    mnum = 1
    #for i in range(len(text)):
    i = 0
    while i < len(text):
        if text[i] == '(':
            txt += 'M' + str(mnum) + ' '
            mnum += 1
            i += 1
            continue
        if i < len(text)-1:
            if text[i].isdigit() and text[i+1].isdigit():
                txt += 'l' + text[i] + text[i+1] + ' '
                i += 2
                continue
        if text[i].isdigit():
            txt += 'l' + text[i] + ' '
        i += 1
    return txt.rstrip()

def branch_switch(tree):
    #print('l', tree.left)
    #print('r', tree.right)
    flag_r_end, flag_l_end = 0, 0
    if len(repr(tree.right)) > 3:
        branch_switch(tree.right)
    else: flag_r_end = 1
    if len(repr(tree.left)) > 3:
        branch_switch(tree.left)
    else: flag_l_end = 1
    if not flag_r_end or not flag_l_end:
        if len(repr(tree.left)) > len(repr(tree.right)):
            #print('change', tree.left, tree.right)
            tree.right, tree.left = tree.left, tree.right

In [11]:
eqglist = pd.read_excel('210706_psy_allgroups_eqg_res_s.xlsx')
#eqglist = pd.read_excel('210805_postcode35_l9_ag.xlsx')
#eqglist = pd.read_excel('210702_arct_allgroups_eqg_res_min.xlsx')
lnum = 10
scale = 4
# better than set lnum - to find it from data

eqglist = eqglist[eqglist.iloc[:,1] <= scale]
eqglist.reset_index(inplace = True, drop = True) 
# sort groups not in scale
eqglist = eqglist.iloc[:,0]

if len(eqglist.iloc[len(eqglist)-1].split()) < lnum-1:
    print("no available structures in input data")
else:
    # add full list structure on the last string
    eqglist[len(eqglist)] = listToString_s([i for i in range(1, lnum+1)])

print("len of data in scale:", len(eqglist))
print("last string:", eqglist.iloc[len(eqglist)-1])
eqglist
#convert string of digits into letters
for i in range(0,len(eqglist)):
    eqglist[i] = digit2alpha(eqglist[i])
eqglist

# need to sort eqglist dy length of group
# проверку вхождения можно делать через in
lol = [[] for i in range(lnum-1)]
for i in range(len(eqglist)):
    lol[len(eqglist[i].split())-2].append(eqglist[i])

len of data in scale: 473
last string: 1 2 3 4 5 6 7 8 9 10


In [9]:
class Node(object):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def __repr__(self):
        return '(%s %s)' % (self.left, self.right)

def add_leaf(tree, label):
    yield Node(label, tree)
    if isinstance(tree, Node):
        for left in add_leaf(tree.left, label):
            yield Node(left, tree.right)
        for right in add_leaf(tree.right, label):
            yield Node(tree.left, right)
    
def enum_unordered(labels):
    if len(labels) == 1:
        yield labels[0]
    else:
        for tree in enum_unordered(labels[1:]):
            for new_tree in add_leaf(tree, labels[0]):
                yield new_tree
                               
def check_if_tree_correct(tree):
    # check if tree in base
    if isinstance(tree, Node):
        one = repr(tree)
        one = re.sub("\(|\)", "", one)
        #one = one.lower()
        one = ' '.join(sorted(one.split()))
        flag_1 = 0
        #print(one)
        one_len = len(one.split())-2
        for i in range(len(lol[one_len])):
            # do search only in list w corresp num of dig
            if lol[one_len][i] == one: 
                flag_1 = 1
                break
        if not flag_1:
            return 0
            #print(space+"nt_l", new_tree)
        left = check_if_tree_correct(tree.left)
        right = check_if_tree_correct(tree.right)
        if left and right: return 1
        else: return 0
    # if not tree - output error
    return 1

        

In [ ]:
# for tree in add_leaf(Node("b", "c"), "a"):
#     print(tree)

start = time.time()
cnt = 0
store_tree = []
for tree in enum_unordered(("a", "b", "c", "d", 'e', "f", "g", "h", "i", 'j')):
#for tree in enum_unordered(("a","b","c","d",'e')):
    if check_if_tree_correct(tree):
        store_tree.append(tree)
        #print(tree)
        cnt += 1
    
print(cnt)
finish = time.time()
result = finish - start
print("Program time: " + str(result) + " seconds.")
for i in range(len(store_tree)):
    print(i, ' ', store_tree[i])

# нужно сделать проверку - если внутри любой скобки только заглавные - выкинуть

In [ ]:
# need to convert notes to the left tree view
for i in range(len(store_tree)):
    branch_switch(store_tree[i])
    print(i, ' ', store_tree[i])

In [14]:
# glimpse
store_tree[12]

((b c) (e ((a f) (g ((d h) (i j))))))

In [21]:
# convert notes to Ml format
store_tree_ml =[]
for i in range(len(store_tree)):
    text1 = alpha2digit(repr(store_tree[i]))
    store_tree_ml.append(braket2ml(text1))

In [22]:
# glimpse
store_tree_ml[0]

'M1 l2 M2 M3 l1 l3 M4 l5 M5 l6 M6 l7 M7 M8 l4 l8 M9 l9 l10'

In [23]:
df = pd.DataFrame(store_tree_ml[0:1048576])
#df = pd.DataFrame(store_tree_ml)
df.to_excel('210819_psy_l10d4_ssml_p1.xlsx', engine="xlsxwriter", index=False)
df = pd.DataFrame(store_tree_ml[1048576:len(store_tree_ml)])
df.to_excel('210819_psy_l10d4_ssml_p2.xlsx', engine="xlsxwriter", index=False)
        